In [ ]:
import load_data 
import network 
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

In [ ]:
path = './data/'
filename = 'dataset_batch_3.mat'
train_input_dataset,train_output_dataset = load_data.load_dataset(path,filename)
print(train_input_dataset.shape,train_output_dataset.shape)

In [ ]:
# path = './data/'
# filename = 'dataset_batch_2.mat'
# test_input_dataset,test_output_dataset = load_data.load_dataset(path,filename)
# print(test_input_dataset.shape,test_output_dataset.shape)
# testing_dataset = load_data.DeconvolutionDataset(test_input_dataset,test_output_dataset,transform=load_data.ToTensor())

In [ ]:
training_dataset = load_data.DeconvolutionDataset(train_input_dataset,train_output_dataset,
                                                  transform=transforms.Compose([load_data.ImageSelect(25),
                                                                              load_data.ToTensor()]))
dataloader = DataLoader(training_dataset, batch_size=80,shuffle=True, num_workers=4)

In [ ]:
device = torch.device("cuda:0")
model = network.FullyConvNet_25()
# model = nn.DataParallel(model)
# model.cuda()
model.to(device)

criterion_2 = nn.MSELoss()
criterion_1 = nn.SmoothL1Loss()

# optimizer = optim.SGD(model.parameters(), lr=1e-9)
optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [ ]:
num_epochs = 10000
for epoch in range(num_epochs):
    for batch_idx, sample in enumerate(dataloader):
        inputs,target = sample['input'].to(device),sample['output'].to(device)
#         inputs,target = sample['input'].cuda(),sample['output'].cuda()
        
#         inputs = Variable(x_train).cuda()
#         target = Variable(y_train).cuda()

        # forward
        out = model(inputs)
        loss = torch.max(criterion_1(out, target),criterion_2(out, target))
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch < 100:
        print('Epoch[{}/{}], loss: {:.6f}'
              .format(epoch+1, num_epochs, loss.data[0]))
    elif (epoch+1) % 100 == 0:
        print('Epoch[{}/{}], loss: {:.6f}'
              .format(epoch+1, num_epochs, loss.data[0]))


In [ ]:
model.eval()

In [ ]:
# print(len(training_dataset),len(testing_dataset))
# print(training_dataset)

In [ ]:
# train_data = np.asarray(np.transpose(train_input_dataset[:100,:,:,:],(0,1,2,3)),dtype=np.float32)
# train_data = torch.from_numpy(train_data)
# predict = model(train_data.to(device))
# predict_train = predict.data.cpu().numpy()

In [ ]:
# print(predict_train.shape)

In [ ]:
# plt.imshow(predict_train[20,0,:,:])

In [ ]:
# plt.imshow(train_output_dataset[20,:,:])

In [ ]:
# plt.imshow(train_input_dataset[20,190,:,:])

In [ ]:
path = './data/'
filename = 'dataset_test.mat'
test_input_dataset,test_output_dataset = load_data.load_dataset(path,filename)

In [ ]:
test_data = np.asarray(np.transpose(test_input_dataset[:4,:25,:,:],(0,1,2,3)),dtype=np.float32)
test_data = torch.from_numpy(test_data)
predict = model(test_data.to(device))
predict_test = predict.data.cpu().numpy()
print(predict_test.shape)

In [ ]:
plt.imshow(predict_test[1,0,:,:])

In [ ]:
plt.imshow(test_output_dataset[1,:,:])

In [ ]:
plt.imshow(test_input_dataset[3,4,:,:])